# AI 논문 공동 저자 관계 분석 (연관 분석)

## 연관 분석 통계량의 AI 연구 협력 맥락에서의 의미
- Support: 전체 논문 중 특정 국가 조합의 연구자들이 공동 저자로 참여하는 비율. 이는 해당 국가들 간의 연구 협력 빈도.
- Confidence: 한 국가의 연구자가 논문을 작성할 때 다른 국가의 연구자와 공동으로 작성할 확률. 이는 연구 협력의 강도.
- Lift: 두 국가 간의 연구 협력이 우연히 발생할 확률 대비 실제로 발생한 비율. 1보다 크면 긍정적인 협력 관계.

In [1]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

pd.set_option('display.max_rows', None)
data = pd.read_csv("./data/ai_research_papers.csv")


/home/pkmon1d/miniconda3/lib/python3.11/site-packages/pandas/io/parsers/c_parser_wrapper.py:365: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  common_type = np.find_common_type(
/home/pkmon1d/miniconda3/lib/python3.11/site-packages/pandas/io/parsers/c_parser_wrapper.py:365: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  common_type = np.find_common_type(
/home/pkmon1d/miniconda3/lib/python3.11/site-packages/pandas/io/parsers/c_parser_wrapper.py:365: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more info

## 전체 데이터 기준

#### 미국과의 광범위한 국제 협력
- 미국은 이 데이터셋에서 가장 많은 연결을 가진 국가로 나타나며, 많은 국가들과의 높은 연관 규칙을 보여줌. 그러나 높은 consequent support에도 불구하고 대부분의 confidence 값은 비교적 낮음(20~40% 범위). 이는 미국이 광범위한 국제 연구 네트워크를 가지고 있지만, 특정 국가와의 연구 협력이 상대적으로 분산되어 있음을 의미.

#### 중국과 홍콩, 싱가포르의 강력한 연구 연결
- 중국과 홍콩, 그리고 싱가포르는 높은 lift 값과 상대적으로 높은 confidence를 가진 규칙을 형성. 
- 특히 홍콩과 중국 간의 연구 협력은 매우 높은 confidence(약 51%)와 lift(약 2.87)를 보여줌. 이는 이 두 지역이 매우 밀접한 연구 협력 관계를 가지고 있음을 나타냄.

#### 유럽 국가들 간의 연구 협력
- 유럽 내에서는 독일, 영국, 네덜란드, 스위스 등이 서로 밀접하게 연결. 이들 국가 간의 규칙은 상대적으로 높은 lift 값을 가짐
- 독일과 영국은 다른 유럽 국가들과도 강한 연구 협력을 보여줌.

#### 아시아-태평양 지역의 연구 협력
- 호주, 중국, 홍콩, 싱가포르 등 아시아-태평양 지역 국가들 간에도 다수의 연구 협력. 
- 호주와 중국 간의 규칙은 높은 lift 값을 가지며, 이는 두 나라 간의 과학 기술 분야에서의 중요한 연결고리를 나타냄.

In [2]:
dataset = []
for rows in data.iterrows() :
    #affiliation_country
    #affilname
    af = rows[1]['affiliation_country']
    af = set(af.split(';'))
    dataset.append(list(af))
len(dataset)

# 데이터를 원-핫 인코딩 형태로 변환
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)
df = df.iloc[:,1:]
frequent_itemsets = apriori(df, min_support=0.001, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.05)

/home/pkmon1d/miniconda3/lib/python3.11/site-packages/pandas/core/dtypes/cast.py:1846: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])  # type: ignore[arg-type]


In [22]:
rules['anc_length'] = rules['antecedents'].apply(lambda x: len(x))
rules['cons_length'] = rules['consequents'].apply(lambda x: len(x))
rules[['antecedents', 'consequents', 'antecedent support',	'consequent support','support', 'confidence', 'lift']].loc[ ((rules['anc_length'] == 1) ) ].sort_values('confidence', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift
14,(Hong Kong),(China),0.030495,0.176952,0.015491,0.507990,2.870776
45,(Israel),(United States),0.027058,0.457650,0.010610,0.392141,0.856857
17,(Singapore),(China),0.024581,0.176952,0.008731,0.355212,2.007387
51,(Saudi Arabia),(United States),0.003356,0.457650,0.001026,0.305831,0.668263
59,(Switzerland),(United States),0.026523,0.457650,0.007639,0.288001,0.629304
0,(Australia),(China),0.035036,0.176952,0.010071,0.287460,1.624509
13,(Canada),(United States),0.054577,0.457650,0.015561,0.285125,0.623019
44,(India),(United States),0.021649,0.457650,0.005829,0.269270,0.588375
52,(Singapore),(United States),0.024581,0.457650,0.005626,0.228898,0.500158
43,(Hong Kong),(United States),0.030495,0.457650,0.006767,0.221913,0.484896


In [23]:
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets[ (frequent_itemsets['length'] == 1) ].sort_values('support', ascending = False).head(20)

,support,itemsets,length
50,0.457650,(United States),1
7,0.176952,(China),1
49,0.075647,(United Kingdom),1
13,0.074628,(Germany),1
5,0.054577,(Canada),1
12,0.046193,(France),1
1,0.035036,(Australia),1
23,0.032577,(Japan),1
15,0.030495,(Hong Kong),1
21,0.027058,(Israel),1


In [24]:
frequent_itemsets[ (frequent_itemsets['length'] >= 2) ].sort_values('support', ascending = False).head(20)

,support,itemsets,length
76,0.037351,"(United States, China)",2
121,0.016647,"(United States, United Kingdom)",2
68,0.015561,"(Canada, United States)",2
71,0.015491,"(Hong Kong, China)",2
95,0.012866,"(Germany, United States)",2
102,0.010610,"(United States, Israel)",2
52,0.010071,"(Australia, China)",2
73,0.008731,"(Singapore, China)",2
86,0.007738,"(France, United States)",2
119,0.007639,"(United States, Switzerland)",2


## 2023년 기준
#### 중국과 아시아 국가들의 강한 연구 협력
- 마카오와 홍콩이 중국과 높은 신뢰도와 향상도 -> 이들 지역의 연구자들이 중국 본토의 연구자들과 빈번하게 협력하고 있음 (지리적 근접성과 문화적, 언어적 유사성)  
- 싱가포르와 중국 간의 협력은 신뢰도가 다소 낮지만 여전히 중요한 연결고리를 가지고 있음.  

#### 유럽 국가들 간의 연구 협력
- 오스트리아와 독일, 폴란드와 독일 등은 높은 향상도를 보이는데, 이는 이들 국가의 연구자들이 서로 간에 강한 학문적 연결고리를 가지고 있음을 의미. 유럽 연구 네트워크의 통합성을 나타내는 결과.
- 스웨덴과 영국, 네덜란드와 영국 간의 연결도 중요하며, 유럽 내에서 영국이 중요한 연구 중심지로서의 역할을 수행하고 있음.

#### 미국과 중동 국가들의 협력
- 사우디 아라비아와 미국, 이스라엘과 미국 간의 연결은 각각의 신뢰도는 다소 낮지만 미국과의 연구 협력이 이들 국가에게 중요함. 
- 이스라엘과 미국 간의 연결은 양국 간의 기술 및 연구 교류가 활발함을 시사.

In [33]:
data_2023 = data.loc[data['year']==2023]
dataset = []
for rows in data_2023.iterrows() :
    #affiliation_country
    #affilname
    af = rows[1]['affiliation_country']
    af = set(af.split(';'))
    dataset.append(list(af))
len(dataset)

# 데이터를 원-핫 인코딩 형태로 변환
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)
df = df.iloc[:,1:]
frequent_itemsets = apriori(df, min_support=0.001, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.05)

/home/pkmon1d/miniconda3/lib/python3.11/site-packages/pandas/core/dtypes/cast.py:1846: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])  # type: ignore[arg-type]


In [34]:
rules['anc_length'] = rules['antecedents'].apply(lambda x: len(x))
rules['cons_length'] = rules['consequents'].apply(lambda x: len(x))
rules[['antecedents', 'consequents', 'antecedent support',	'consequent support','support', 'confidence', 'lift']].loc[ ((rules['anc_length'] == 1) ) ].sort_values('confidence', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift
26,(Macao),(China),0.003823,0.336957,0.002810,0.734940,2.181108
22,(Hong Kong),(China),0.056014,0.336957,0.039108,0.698191,2.072047
29,(Singapore),(China),0.039246,0.336957,0.021650,0.551643,1.637132
0,(Australia),(China),0.044221,0.336957,0.020959,0.473958,1.406584
79,(Saudi Arabia),(United States),0.003823,0.361371,0.001704,0.445783,1.233589
64,(Israel),(United States),0.021742,0.361371,0.008890,0.408898,1.131520
91,(United Arab Emirates),(United States),0.006633,0.361371,0.002672,0.402778,1.114583
62,(India),(United States),0.023539,0.361371,0.007647,0.324853,0.898947
19,(Canada),(United States),0.047952,0.361371,0.015339,0.319885,0.885198
89,(Switzerland),(United States),0.031185,0.361371,0.009673,0.310192,0.858376


In [35]:
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets[ (frequent_itemsets['length'] == 1) ].sort_values('support', ascending = False).head(20)

,support,itemsets,length
46,0.361371,(United States),1
7,0.336957,(China),1
12,0.081072,(Germany),1
45,0.074854,(United Kingdom),1
14,0.056014,(Hong Kong),1
5,0.047952,(Canada),1
0,0.044221,(Australia),1
34,0.039246,(Singapore),1
11,0.036943,(France),1
36,0.036667,(South Korea),1


In [125]:
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets[ (frequent_itemsets['length'] >= 2) ].sort_values('support', ascending = False)

,support,itemsets,length
85,0.053388,"(China, United States)",2
74,0.039108,"(China, Hong Kong)",2
80,0.021650,"(China, Singapore)",2
49,0.020959,"(China, Australia)",2
141,0.017642,"(United States, United Kingdom)",2
...,...,...,...
126,0.001013,"(United States, Poland)",2
114,0.001013,"(United States, Ireland)",2
60,0.001013,"(United States, Belgium)",2
57,0.001013,"(Italy, Austria)",2


## 2013년 기준

In [36]:
data_2013 = data.loc[data['year']==2013]
dataset = []
for rows in data_2013.iterrows() :
    #affiliation_country
    #affilname
    af = rows[1]['affiliation_country']
    af = set(af.split(';'))
    dataset.append(list(af))
len(dataset)

# 데이터를 원-핫 인코딩 형태로 변환
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)
df = df.iloc[:,1:]
frequent_itemsets = apriori(df, min_support=0.001, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.05)

/home/pkmon1d/miniconda3/lib/python3.11/site-packages/pandas/core/dtypes/cast.py:1846: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])  # type: ignore[arg-type]


In [43]:
rules['anc_length'] = rules['antecedents'].apply(lambda x: len(x))
rules['cons_length'] = rules['consequents'].apply(lambda x: len(x))
rules[['antecedents', 'consequents', 'antecedent support',	'consequent support','support', 'confidence', 'lift']].loc[ ((rules['anc_length'] == 1) & (rules['lift'] > 1) ) ].sort_values('lift', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift
8,(Hong Kong),(China),0.016910,0.020616,0.002085,0.123288,5.980145
9,(China),(Hong Kong),0.020616,0.016910,0.002085,0.101124,5.980145
10,(Singapore),(China),0.014362,0.020616,0.001158,0.080645,3.911743
11,(China),(Singapore),0.020616,0.014362,0.001158,0.056180,3.911743
1,(Australia),(Hong Kong),0.024786,0.016910,0.001621,0.065421,3.868775
2,(Hong Kong),(Australia),0.016910,0.024786,0.001621,0.095890,3.868775
3,(Sweden),(Australia),0.016215,0.024786,0.001158,0.071429,2.881842
13,(Denmark),(United Kingdom),0.007413,0.069724,0.001390,0.187500,2.689161
23,(Ireland),(United Kingdom),0.006254,0.069724,0.001158,0.185185,2.655962
18,(Netherlands),(Germany),0.013667,0.052814,0.001621,0.118644,2.246432


In [38]:
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets[ (frequent_itemsets['length'] == 1) ].sort_values('support', ascending = False).head(20)

,support,itemsets,length
52,0.451409,(United States),1
8,0.120652,(China),1
51,0.086625,(United Kingdom),1
17,0.081158,(Germany),1
6,0.060326,(Canada),1
16,0.057970,(France),1
2,0.033085,(Australia),1
25,0.032614,(Italy),1
26,0.032425,(Japan),1
40,0.028655,(Singapore),1


## 2003년 기준

In [39]:
data_2003 = data.loc[data['year']==2003]
dataset = []
for rows in data_2003.iterrows() :
    #affiliation_country
    #affilname
    af = rows[1]['affiliation_country']
    af = set(af.split(';'))
    dataset.append(list(af))
len(dataset)

# 데이터를 원-핫 인코딩 형태로 변환
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)
df = df.iloc[:,1:]
frequent_itemsets = apriori(df, min_support=0.001, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.05)

/home/pkmon1d/miniconda3/lib/python3.11/site-packages/pandas/core/dtypes/cast.py:1846: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])  # type: ignore[arg-type]


In [40]:
rules['anc_length'] = rules['antecedents'].apply(lambda x: len(x))
rules['cons_length'] = rules['consequents'].apply(lambda x: len(x))
rules[['antecedents', 'consequents', 'antecedent support',	'consequent support','support', 'confidence', 'lift']].loc[ ((rules['anc_length'] == 1) ) ].sort_values('confidence', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift
25,(Israel),(United States),0.038916,0.615242,0.015752,0.404762,0.657890
37,(Taiwan),(United States),0.007644,0.615242,0.002085,0.272727,0.443285
5,(Brazil),(United States),0.007876,0.615242,0.002085,0.264706,0.430247
27,(Italy),(United States),0.027334,0.615242,0.006949,0.254237,0.413231
7,(Canada),(United States),0.056752,0.615242,0.014130,0.248980,0.404686
31,(Singapore),(United States),0.014362,0.615242,0.003475,0.241935,0.393236
30,(Netherlands),(United States),0.013667,0.615242,0.003243,0.237288,0.385683
34,(Spain),(United States),0.012045,0.615242,0.002780,0.230769,0.375087
35,(Sweden),(United States),0.016215,0.615242,0.003706,0.228571,0.371515
32,(South Korea),(United States),0.011350,0.615242,0.002548,0.224490,0.364880


In [41]:
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets[ (frequent_itemsets['length'] == 1) ].sort_values('support', ascending = False).head(20)

,support,itemsets,length
39,0.615242,(United States),1
38,0.069724,(United Kingdom),1
4,0.056752,(Canada),1
12,0.052814,(Germany),1
18,0.038916,(Israel),1
11,0.038684,(France),1
20,0.031272,(Japan),1
19,0.027334,(Italy),1
0,0.024786,(Australia),1
6,0.020616,(China),1


## 기타 코드

In [4]:
import pandas as pd

# 가정: 데이터 프레임은 'data'라는 변수에 저장되어 있음


# 각 국가별 단독 연구 비율을 계산하는 함수
def calculate_solo_research_rate(data):
    solo_research_rates = {}
    for country in data.columns:
        # 해당 국가가 참여한 모든 논문
        country_participations = data[country]
        
        # 단독으로 참여한 논문
        solo_research = data[(data[country] == True) & (data.drop(columns=[country]).sum(axis=1) == 0)]
        
        # 단독 연구의 비율 계산
        if country_participations.sum() > 0:
            solo_research_rate = len(solo_research) / country_participations.sum()
        else:
            solo_research_rate = 0  # 참여한 논문이 없는 경우
        
        solo_research_rates[country] = solo_research_rate

    return solo_research_rates

# 각 국가별 단독 연구 비율을 계산
solo_research_rates = calculate_solo_research_rate(df)

# 결과 출력
for country, rate in solo_research_rates.items():
    print(f"{country}: {rate:.2%}")


Algeria: 48.67%
Angola: 0.00%
Anguilla: 0.00%
Argentina: 35.71%
Armenia: 38.89%
Australia: 37.48%
Austria: 38.57%
Azerbaijan: 50.00%
Bahamas: 100.00%
Bahrain: 18.18%
Bangladesh: 49.21%
Belarus: 33.33%
Belgium: 39.67%
Benin: 100.00%
Bhutan: 100.00%
Bolivia: 20.00%
Bosnia and Herzegovina: 53.85%
Botswana: 60.00%
Brazil: 61.95%
Brunei Darussalam: 0.00%
Bulgaria: 29.33%
Burkina Faso: 0.00%
Cambodia: 0.00%
Cameroon: 45.45%
Canada: 45.42%
Chad: 0.00%
Chile: 31.76%
China: 56.05%
Colombia: 35.57%
Comoros: 0.00%
Congo: 0.00%
Costa Rica: 59.09%
Cote d'Ivoire: 0.00%
Croatia: 51.22%
Cuba: 19.23%
Cyprus: 37.57%
Czech Republic: 41.17%
Denmark: 35.39%
Dominica: 0.00%
Dominican Republic: 0.00%
Ecuador: 27.45%
Egypt: 45.21%
El Salvador: 23.08%
Estonia: 47.24%
Ethiopia: 18.18%
Fiji: 33.33%
Finland: 44.43%
France: 46.71%
Georgia: 0.00%
Germany: 52.27%
Ghana: 11.11%
Greece: 43.99%
Grenada: 0.00%
Guam: 20.00%
Guatemala: 40.00%
Haiti: 0.00%
Honduras: 54.55%
Hong Kong: 25.11%
Hungary: 43.33%
Iceland: 23.64%


In [3]:
import pandas as pd

# 가정: 데이터 프레임은 'data'라는 변수에 저장되어 있음

# 특정 국가와 공동으로 연구한 국가별 비율을 계산하는 함수
def calculate_cooperation_rates(data, target_country):
    if target_country not in data.columns:
        return f"Error: {target_country} is not a valid country in the dataset."
    
    # 해당 국가가 참여한 모든 논문 필터링
    cooperation_data = data[data[target_country] == True]
    
    # 공동 연구 논문 수 (해당 국가 제외)
    total_cooperations = cooperation_data.drop(columns=[target_country]).sum().sum()
    
    # 각 국가와의 공동 연구 비율 계산
    cooperation_rates = {}
    for country in data.columns:
        if country != target_country:
            # 해당 국가와 공동으로 참여한 논문 수
            country_cooperations = cooperation_data[country].sum()
            
            # 비율 계산
            if total_cooperations > 0 and country_cooperations > 0:
                cooperation_rate = country_cooperations / total_cooperations
                cooperation_rates[country] = cooperation_rate

    # 0%를 제외하고 내림차순으로 정렬
    sorted_cooperation_rates = dict(sorted(cooperation_rates.items(), key=lambda item: item[1], reverse=True))

    return sorted_cooperation_rates

# 사용자로부터 국가 이름 입력받기
target_country = 'China'

# 공동 연구 국가별 비율 계산
cooperation_rates = calculate_cooperation_rates(df, target_country)

# 결과 출력
print(f"Co-research rates for {target_country}:")
for country, rate in cooperation_rates.items():
    print(f"{country}: {rate:.2%}")


Co-research rates for China:
United States: 37.93%
Hong Kong: 15.73%
Australia: 10.23%
Singapore: 8.87%
United Kingdom: 6.04%
Canada: 4.67%
Japan: 2.35%
Germany: 1.78%
Switzerland: 1.42%
France: 1.08%
South Korea: 0.85%
United Arab Emirates: 0.85%
Netherlands: 0.73%
Macao: 0.72%
Saudi Arabia: 0.59%
Italy: 0.54%
Israel: 0.54%
Denmark: 0.53%
Taiwan: 0.50%
Finland: 0.44%
Spain: 0.35%
Sweden: 0.30%
India: 0.26%
New Zealand: 0.25%
Austria: 0.24%
Ireland: 0.23%
Belgium: 0.21%
Norway: 0.19%
Qatar: 0.19%
Czech Republic: 0.10%
Malaysia: 0.10%
Greece: 0.10%
Luxembourg: 0.09%
Pakistan: 0.08%
Brazil: 0.06%
Iran: 0.06%
Poland: 0.06%
Russian Federation: 0.05%
Philippines: 0.04%
Puerto Rico: 0.04%
Bangladesh: 0.04%
Hungary: 0.03%
Portugal: 0.03%
Slovenia: 0.03%
Viet Nam: 0.03%
Tunisia: 0.03%
Argentina: 0.03%
Egypt: 0.03%
Thailand: 0.03%
Chile: 0.02%
Cyprus: 0.02%
South Africa: 0.02%
Turkey: 0.02%
Algeria: 0.01%
Indonesia: 0.01%
Mexico: 0.01%
Oman: 0.01%
Romania: 0.01%
Serbia: 0.01%
Ukraine: 0.01%
Col

In [59]:
# 사용자로부터 국가 이름 입력받기
target_country = 'Austria'

# 공동 연구 국가별 비율 계산
cooperation_rates = calculate_cooperation_rates(df, target_country)

# 결과 출력
print(f"Co-research rates for {target_country}:")
for country, rate in cooperation_rates.items():
    print(f"{country}: {rate:.2%}")

Co-research rates for Austria:
United States: 16.97%
Germany: 15.75%
United Kingdom: 9.89%
France: 5.95%
Switzerland: 4.79%
Italy: 4.67%
Canada: 3.10%
Netherlands: 3.10%
Australia: 3.02%
Czech Republic: 2.73%
Spain: 2.44%
Israel: 2.32%
Sweden: 2.29%
China: 1.89%
Japan: 1.83%
Denmark: 1.77%
India: 1.65%
Finland: 1.39%
Poland: 1.36%
Belgium: 1.19%
Norway: 1.04%
Saudi Arabia: 0.87%
Greece: 0.67%
Singapore: 0.64%
Hong Kong: 0.61%
Russian Federation: 0.52%
Chile: 0.49%
Ireland: 0.49%
Iran: 0.46%
Portugal: 0.44%
New Zealand: 0.41%
Turkey: 0.41%
Brazil: 0.38%
Slovenia: 0.35%
Estonia: 0.29%
South Korea: 0.29%
Hungary: 0.26%
Pakistan: 0.26%
Romania: 0.26%
Uruguay: 0.26%
Indonesia: 0.23%
South Africa: 0.20%
Taiwan: 0.20%
Argentina: 0.17%
Luxembourg: 0.17%
Mexico: 0.15%
Egypt: 0.12%
Namibia: 0.12%
Slovakia: 0.12%
Qatar: 0.09%
United Arab Emirates: 0.09%
Bangladesh: 0.06%
Bulgaria: 0.06%
Colombia: 0.06%
Costa Rica: 0.06%
Cyprus: 0.06%
Georgia: 0.06%
Iceland: 0.06%
Malaysia: 0.06%
Thailand: 0.06%
U